In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [74]:
#Retrieve data from web location and transform csv in dataframe
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
#specific_site= df[df['Launch Site'] == 'CCAFS LC-40']
#specific_site = df.loc[df['Launch Site'] == 'CCAFS LC-40']
#specific_site
max_payload = df['Payload Mass (kg)'].max()
min_payload = df['Payload Mass (kg)'].min()
#df_payload= df[(df['Payload Mass (kg)']> 500) & (df['Payload Mass (kg)']<= 9600)]
#df_payload

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
2,2,3,CCAFS LC-40,0,525.00,F9 v1.0 B0005,v1.0
4,4,5,CCAFS LC-40,0,677.00,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,0,3170.00,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,0,3325.00,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,0,2296.00,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,0,1316.00,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,0,4535.00,F9 v1.1,v1.1
10,10,12,CCAFS LC-40,0,4428.00,F9 v1.1 B1011,v1.1
11,11,13,CCAFS LC-40,0,2216.00,F9 v1.1 B1010,v1.1
12,12,14,CCAFS LC-40,0,2395.00,F9 v1.1 B1012,v1.1


In [ ]:
# Create a dash application
app = Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
                                # TASK 1: Add a Launch Site Drop-down Input Component
                                html.H1('SpaceX Launch Records Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 40}),
                                
                                dcc.Dropdown(id='site_dropdown',
                                options=['All Sites','CCAFS LC-40','VAFB SLC-4E','KSC LC-39A','CCAFS SLC-40'],
                                value='All Sites', #By default, input value for dropdown is 'All'
                                placeholder="Select a Launch Site here",),
                                html.Br(),
                                
                                # TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
                                html.Div(dcc.Graph(id='success_pie_chart')),
                                html.Br(),
    
                                #TASK 3: Add a Range Slider to Select Payload
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload_slider',
                                                min=0,max=10000,step=1000,
                                                value=[min_payload,max_payload],
                                                marks={0: '0', 2500:'2500',5000:'5000',
                                                7500:'7500', 10000: '10000'}),
                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success_payload_scatter_chart')),
                                ]
                     )

# Callback for dropdown list input
@app.callback(
    Output('success_pie_chart', 'figure'), #Output(component_id, component_property)
    Input('site_dropdown', 'value') #Input(component_id, component_property)
             )
def pie_chart(site_dropdown):
    if site_dropdown == 'All Sites':
        piechart = px.pie(df, names='Launch Site', values='class', title='Total Launches for All Sites') #piechart is based on number of 1 in 'class' and value = percentage of 1 for each launch site
        return piechart
    else:
        specific_site= df[df['Launch Site'] == site_dropdown]
        piechart = px.pie(specific_site, names='class', title=f'Percentage of successful launches from {site_dropdown}') #piechart has no value but has category/names which is 1 or 0
        return piechart
    
    
    
@app.callback(
    Output('success_payload_scatter_chart', 'figure'), #Output(component_id, component_property)
    [Input('site_dropdown', 'value'), #Input(component_id, component_property)
    Input('payload_slider', 'value')] #Input(component_id, component_property)
              )
def scatter_plot(site_dropdown, payload_slider):
    if site_dropdown == 'All Sites':
        df_payload= df[(df['Payload Mass (kg)']> payload_slider[0]) & (df['Payload Mass (kg)']<= payload_slider[1])]
        scatterplot = px.scatter(df_payload, x='Payload Mass (kg)', y='class', title='Correlation between Payload and Success for all Sites', color="Booster Version Category")
        return scatterplot  
    else:
        df_payload_specific= df[df['Launch Site'] == site_dropdown]
        df_payload_specific= df_payload_specific[(df_payload_specific['Payload Mass (kg)']> payload_slider[0]) & (df_payload_specific['Payload Mass (kg)']<= payload_slider[1])]
        scatterchart = px.scatter(df_payload_specific, x='Payload Mass (kg)', y='class', title='Correlation between Payload and Success for all Sites', color="Booster Version Category")
        return scatterplot 

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2022 17:35:54] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2022 17:35:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 17:35:54] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2022 17:35:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 17:35:54] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2022 17:35:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 17:35:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2022 17:35:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 17:35:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2022 17:35:54] "GET /_dash-com

In [4]:
app = Dash(__name__)


app.layout = html.Div([
    html.H4('Analysis of the restaurant sales'),
    dcc.Graph(id="graph"),
    html.P("Names:"),
    dcc.Dropdown(id='names',
        options=['smoker', 'day', 'time', 'sex'],
        value='day', clearable=False
    ),
    html.P("Values:"),
    dcc.Dropdown(id='values',
        options=['total_bill', 'tip', 'size'],
        value='total_bill', clearable=False
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("names", "value"), 
    Input("values", "value"))
def generate_chart(names, values):
    df_ = px.data.tips() # replace with your own data source
    fig = px.pie(df_, values=values, names=names, hole=.3)
    return fig

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
